# Проект: Retrieval-система по статьям с arXiv


## Постановка задачи

В проекте вам предстоит построить retrieval-систему и оценить её качество. У вас будет датасет с аннотациями статей с портала arXiv.org и тестовый набор запросов и правильных ответов к ним. Также выполнить профилирование частей модели.

## Критерии успеха
Целевая метрика — качество поисковой выдачи: Mean Reciprocal Rank (MRR@5).

Проект считается выполненным при **MRR@5 > 0.91**.

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных
Дан архив с датасетом.

В нём есть метаданные статей `arxiv-metadata-s.json` и набор тестовых запросов для оценки качества поиска `test_sample.csv`.
- `test_sample.csv` имеет структуру:
    - `query` — запрос к поисковой системе,
    - `id` — ID статьи, отвечающей на запрос query (один запрос — один верный ответ),
    - `abstract` — аннотация статьи id.
- arxiv-metadata-s.json имеет структуру:
    - `id` — ID статьи,
    - `abstract` — аннотация статьи,
    - `title` — название статьи.

## Инициализация

### Импорт библиотек